In [ ]:
#Code adapted from Professor Wloka from CS153 transfer_learning demo
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
import copy

plt.rcParams['figure.figsize'] = [16, 10]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(), # randomly flip with a 0.5 chance
        transforms.RandomVerticalFlip(), # randomly flip with a 0.5 chance
        #randomly change the brightness with a max of 0.5 and the hue with a range of -.4 to .4
        transforms.ColorJitter(0.05, 0, 0,(-0.4, 0.4)), 
        transforms.RandomGrayscale(0.2), # randomly change image to grayscale with a 0.2 chance
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.CenterCrop(224),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'project_data_c'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

#creates a txt file with all of the different classes
with open("class_names.txt", 'w') as f:
    for classes in class_names:
        f.write(str(classes) + '\n')

In [ ]:
def tensor_show(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

tensor_show(out, title=[class_names[x] for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    nb_classes = len(class_names)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0


    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_corrects_top3 = 0
            #creates a confusion matrix
            confusion_matrix = torch.zeros(nb_classes, nb_classes)

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                labels = labels.to(device)
                inputs = inputs.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)  
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                #calculates top 3 accuracies credit to weiaicunzai github:https://gist.github.com/weiaicunzai/2a5ae6eac6712c70bde0630f3e76b77b
                maxk = 3  # labels that we will consider correct
                _, y_pred = outputs.topk(k=maxk, dim=1)
                y_pred = y_pred.t()
                target_reshaped = labels.view(1, -1).expand_as(y_pred)
                correct = (y_pred == target_reshaped)
                # get tensor of which topk answer was right
                ind_which_topk_matched_truth = correct[:3]
                # flatten it to help compute if we got it correct for each example in batch
                flattened_indicator_which_topk_matched_truth = ind_which_topk_matched_truth.reshape(-1).float()
                # adds if there is a correct predicion
                running_corrects_top3 += flattened_indicator_which_topk_matched_truth.float().sum(dim=0, keepdim=True)


                #populate the confusion array
                for t, p in zip(labels.view(-1), preds.view(-1)):
                        confusion_matrix[t.long(), p.long()] += 1


            if phase == 'train':
                scheduler.step()

            running_corrects_top3 = running_corrects_top3.numpy()
            epoch_acc_top3 = running_corrects_top3[0] / dataset_sizes[phase]
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            tp_and_fn = confusion_matrix.sum(1)
            tp_and_fp = confusion_matrix.sum(0)
            tp = confusion_matrix.diagonal()

            precision = tp / tp_and_fp
            recall = tp / tp_and_fn

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Top3 Acc: {epoch_acc_top3:.4f}')
            print(confusion_matrix.diag()/confusion_matrix.sum(1))
            print(f'Precision: {precision} ')
            print(f'Recall: {recall}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_acc_top3 = epoch_acc_top3
                best_model_wts = copy.deepcopy(model.state_dict())
                best_confusion_matrix = confusion_matrix
                best_precision = precision
                best_recall = recall

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f} Best val Acc Top3: {best_acc_top3:.4f}')
    print(f'Best classes Acc: {best_confusion_matrix.diag()/best_confusion_matrix.sum(1)}')
    print(f'Best classes precision: {best_precision}')
    print(f'Best class recall: {best_recall}')
    print(best_confusion_matrix)

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                tensor_show(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

In [ ]:
visualize_model(model_conv)

In [ ]:
def save(filename):
        """saves the model returned from after training it
        inputs: filename the name you want to save the model as
        """
        torch.save(model_conv, filename+".pt")

In [ ]:
def predict(modelname, class_names, image):
    """gives the top 3 predictions of what the image is along with their Probabilities
    input: model the location of the model you want to use,
    class_names location of the text files of the classes the model was trained on
    image: the location of the images you want to predict
    """

    #transforms to be preformed on the input images
    data_transforms = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.CenterCrop(224),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image)
    image = data_transforms(image)
    image = image.unsqueeze(0)

    #save the classes in the class_names into a list
    with open(class_names, 'r') as f:
        class_names = [line.rstrip('\n') for line in f]
    
    
    # Disable grad
    with torch.no_grad():
        
        # Loading the saved model
        model = torch.load(modelname)
        model.eval()
        
        outputs = model(image)
        #find the probaility of each output
        predicted_classes = torch.nn.functional.softmax(outputs, dim=1)
        predicted_classes = predicted_classes.numpy()
        predicted_classes_unsorted = copy.deepcopy(predicted_classes)
        #sort the array so we can find the top predictions
        predicted_classes.sort()

        top3 = []
        #finds out what what the top 3 classes are (theres probability a simpler way to do this)
        i = -1
        while i != -4:
            for j in range(len(predicted_classes[0])):
                if predicted_classes[0][i] == predicted_classes_unsorted[0][j]:
                    top3 += [j]
                    break
            i -=1

        #unnormalize the image to display
        mean = torch.tensor([0.485, 0.456, 0.406])
        std = torch.tensor([0.229, 0.224, 0.225])
        image = image *std[:, None, None] + mean[:, None, None]
        image = image.squeeze(0)
        image = image.T
        image = image.numpy()
        
        # Show result
        plt.imshow((image * 255).astype(np.uint8))
        plt.title(f'Prediction: {class_names[top3[0]]} Probability: {predicted_classes[0][-1]:.4f} \nPrediction: {class_names[top3[1]]} Probability: {predicted_classes[0][-2]:.4f} \nPrediction: {class_names[top3[2]]} Probability: {predicted_classes[0][-3]:.4f}')
        plt.show()